In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, num_warmup=2000,
num_samples=2000, output_format=:mcmcchain);

=====> /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12



Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, CmdStanDir=CMDSTAN_HOME);


make: `/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1' is up to date.
Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan' at line 15)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.


Calling /Users/rob/Projects/StanSupport/cmdstan/bin/stansummary to infer across chains.

Inference for Stan model: m12_6_1_model
4 chains: each with iter=(2000,2000,2000,2000); warmup=(0,0,0,0); thin=(1,1,1,1); 8000 iterations saved.

Warmup took (0.98, 0.89, 0.96, 0.92) seconds, 3.7 seconds total
Sampling took (1.1, 1.1, 1.1, 1.0) seconds, 4.4 seconds total

                    Mean     MCSE  

Describe the draws

In [6]:
describe(chn)

Iterations = 1:2000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 2000

Empirical Posterior Estimates:
                   Mean            SD         Naive SE       MCSE         ESS   
         lp__  927.797842000  3.0401496924 0.03398990687 0.0731388272 1727.80192
accept_stat__    0.948024024  0.0721234576 0.00080636477 0.0009050823 2000.00000
   stepsize__    0.049720925  0.0036726082 0.00004106101 0.0004131749   79.00988
  treedepth__    5.937625000  0.4424464140 0.00494670129 0.0096180930 2000.00000
 n_leapfrog__   76.472500000 36.0983189336 0.40359147504 0.7617987022 2000.00000
  divergent__    0.000000000  0.0000000000 0.00000000000 0.0000000000        NaN
     energy__ -921.303251625  3.9900219928 0.04460980204 0.0936280534 1816.09512
            a    1.070102013  0.7533674328 0.00842290396 0.0176583025 1820.18605
           bp    0.263828642  0.0815373194 0.00091161494 0.0019481822 1751.67571
  a_society.1   -0.196855184  0.2414843084 0.00269987665 0.0043127315 2000

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*